# Getting the climate and soil data

Now that we have initial parameters for our species, we need to define the landscape in which the calibration will take place.

To adapt to the Python functions I've made for these Jupyter Notebooks (see [here](./functionsForCalibration.py)), simulations will take place on a single cell. 

What we need is :

- The climate file (containing all of the climate variables need by PnET)
- The ecoregion file and PnET ecoregion parameters (to decide which ones will be used during the calibration)
- Initial communities files (if needed)

## The climate file

See [Gustafson and Miranda (2023)](./ReferencesAndData/Documentation/Gustafson2024PnETUserGuide.pdf) for detailed information about what the climate file used in PnET simulations must contain.

[Gustafson and Miranda (2023)](./ReferencesAndData/Documentation/Gustafson2024PnETUserGuide.pdf) recommends using a constant climate for calibrating (p. 69). Gustafson also recommends using an "ideal" climate at some points, but deciding what is an ideal climate for a given species is a difficult proposition. As we are going to compare the growth curves generated by PnET Succession with the growth curves estimated from data of the National Forest Inventory (NFI) of Canada (see [](./5.Other_important_data_before_starting.ipynb)), the idea will be rather to get long-term monthly averages (as recommanded by [Gustafson and Miranda (2023)](./ReferencesAndData/Documentation/Gustafson2024PnETUserGuide.pdf) p. 73) in a region that is pretty representative climate of the study area of interest.

PnET Succession requires 5 climate variables to function :

- Maximum Monthly temperature (°C)
- Minimum Monthly temperature (°C)
- Photosynthetically Active Radiation (umol/m2/s)
- Sum of precipitations during the month (mm)
- Mean monthly atmospheric CO2 concentration (ppm)

### Climate data location for the calibration

Since we're doing calibration simulation here, we don't have a particular place where our simulation take place. It's up to us to choose where the climate data comes from, and there is no a "best" way to do things.

It's surely most likely better to use climate conditions that are closer to the average of the conditions we want to simulate in other LANDIS-II simulation rather than to extremes (especially for this first step of calibration). We also want to make sure that the location we use ca be inputted in FVSon.

I propose that we use an area near the border between the boral and temperate forest, and near the center of Ontario. It will be arbitrary.

Here is a map from of the forests of Ontario from the 2016 forest report of Ontario :

![](https://files.ontario.ca/1a-forestregion-map_e_1.png)

The city of [Chapleau](https://www.openstreetmap.org/#map=12/47.8416/-83.4106) seems to be located near the limit between the boreal and temperate forest. I created a simple shapefile polygon around the city that will serve to clip the climate data we want. See the next cell for a map.

In [29]:
# Displays a map where the shapefile that defines the boundaries of the climate data we wanna use is
import geopandas as gpd
import folium
from folium.features import GeoJsonPopup, GeoJsonTooltip

# Load the shapefile
# Replace 'path_to_shapefile.shp' with your actual shapefile path
gdf = gpd.read_file("./ReferencesAndData/SpatialBoundaries/ChapleauBoundariesClimate.shp")

# Check the CRS (Coordinate Reference System) of the shapefile
# If not in WGS84 (EPSG:4326), reproject it
if gdf.crs != 'EPSG:4326':
    gdf = gdf.to_crs('EPSG:4326')

# Create a map centered on the mean of the shapefile bounds
center_lat = gdf.unary_union.centroid.y
center_lon = gdf.unary_union.centroid.x
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add the GeoJSON data to the map with some styling
folium.GeoJson(
    gdf,
    name='Polygons',
    style_function=lambda x: {
        'fillColor': '#ebcb8b',
        'color': '#2e3440',
        'weight': 1,
        'fillOpacity': 0.4
    }
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Alternatively, you can use this simpler approach which works in newer versions of JupyterLab
display(m)

### Climate data source for 🌡️ temperature, 🌧️ precipitation and ☀ solar radiation

#### Choosing the right source - going with ESGF data

There are many potential sources of climate data. I've had to juggle with many of them before finding something that fitted our objective here, but also looking more at the long term of the DIVERSE project.

I first wanted to use data from Climate Canada, as they produce outputs of several Global Climate models from CMIP6 at a very small resolution (downscaled), and accessible through the OpenDAP protocol (which allow us to only download the chunks of the data we need). However, their model lacked some of the variables needed for PnET-Succession, especially the solar radiation. This required to use statistical models to predict this variable based on other datasets, which was really not ideal. In addition, their data was monthly, and we were going to require daily data to accomodate some LANDIS-II extensions we were going to use with PnET-Succession further down the road (e.g. BFOLDS Fire).

I then went on to use data from the ESGF (Earth System Grid Federation); first manually from their "[Metagrid](https://esgf.github.io/nodes.html)" online interface, and then via scripts using their [intake-esgf](https://github.com/esgf2-us/intake-esgf) python package. This allowed us to access outputs from a very large array of GCMs, different variables, etc, at a daily scale, and with the OpenDAP protocol. However, the gridded data here is 100 x 100km of resolution, which is pretty coarse.

I will keep on going with this data from the ESGF in this notebook, so that it is easily re-usable for research teams in other parts of the world (since data from the ESGF covers all of the world). However, for the DIVERSE project, we will most likely use the BioSIM canadian model to downscale climate data, and specially get different climate data for areas with a slope/aspect that can affect local temperature and solar radiation that is received by trees.

#### Climate model that we are using here

There are many Global Climate Models (GCMs), and it's sometimes difficult to choose the right one. Here, I've choosen to go with TaiESM1, which was the closest to the median accross models from CMIP6 for the temperature and precipitations for our first tests with the CanDCS-M6 dataset. According to the litterature, TaiESM1 also seems to have a pretty good performance overall when compared in GCM ensembles (see [here](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2020MS002353)). TaiESM1 also have a lot of data for different SSPs scenarios on the ESGF data repositories, which will be useful for the later steps of the calibration, but also for the DIVERSE project.

:::{tip} Why we are using a single climate model (instead of several together) here
Climate scientists often recommand to use the results from several climate models (GCMs) at once so as to explore the influence of the variability in results that exists between models, and the incertainty that results from it.

Here, I choose to use the results from a single climate model for several reasons :

- The goal of this calibration is not to explore climate change predictions, but simply to calibrate the growth and dynamic of our tree species. For that, we will use historical data simulated by the model we chose (TaiESM1).
- This calibration is already quite complex, and I'd like to keep it as simple as possible.
- Using several models at once requires one of two things : either using a single climate file whose values are averaged along the values simulated by the different GCMs, which results in very, very "mild" climate conditions (since all extreme variations are not synchronized between models); or to use several climate files and create several PnET-Succession growth curves at each point (which makes things quite complicated).

However, once you have calibrated PnET Succession and will use it in your LANDIS-II simulations, it might be good to generate different climate files based on different climate models in order to create "climate replicates" for your LANDIS-II simulation that will allow you to generate an envellope of variability/uncertainty on your LANDIS-II results. To that end, you can use the scripts of this page to generate several climate files for different climate models available in ESGF (be warned that not all models have all of the variables you will need for all of the climate scenario you will want to explore). You can also use the functions from the Xclim python package that will allow you to select a smaller number of models that will represent your climate data properly (see [here](https://xclim.readthedocs.io/en/stable/apidoc/xclim.ensembles.html#xclim.ensembles._reduce.kmeans_reduce_ensemble); you will need to create a large ensemble of climate data to use these functions).
:::

#### Extraction of 🌡️ temperature, 🌧️ precipitation and ☀ solar radiation data for Ontario, 1950-2100 from [ESGF](https://esgf.github.io/nodes.html)

The goal here is to extract the data for :

- The TaiESM1 model
- 4 variables (Tmax, Tmin, precip, PAR)
- Historical range (1950-2010)
- At a daily frequency
- At a 100x100km resolution, but only inside our polygon (see above), and only for a single cell that is closest to the polygon centroid in the end (since we will only simulate one cell at a time during the calibration, we don't need more than one climate stream for one ecoregion)

We extract the raw variables first for each, who each have particular names for CMIP6 data like this one :
- tasmax = maximum daily temperature in kelvin
- tasmin = minimum daily temperature in kelvin
- prc = daily precipitations in kg m-2 s-1
- rsds = average daily downwelling shortwave solar radiation in W/m2

To be certain of these units, we can check the metadata/attributes of the objects manipulated during this script, or on the ESGF metagrid.

Other variables are available at a daily frequency that can be used for other LANDIS-II extensions if you need them. For example, the variables necessary for BFOLDS fire :

- hur, the relative humidty in percentage
- ua, the eastward wind speed in m/s
- va, the northward wind speed in m/s

For other variables codes, see sites such as [this one](https://pcmdi.llnl.gov/mips/cmip3/variableList.html).

In [3]:
# Here, we get the URLs to download the data we need through the OpenDAP protocol,
# by looking at the datasets available in the ESGF nodes
# WARNING : if you have a VPN on, this might result in error as nodes might refuse the connection.
# Simply disable the VPN and try again.

from functionsForCalibration import *
from intake_esgf import ESGFCatalog
import intake_esgf

# Initialize catalog
cat = ESGFCatalog()

# Search for the variables we need for TaiESM1, historical data, and daily frequency
# See cell above for infos about the variables
cat.search(
    experiment_id="historical",
    source_id="TaiESM1",
    frequency="day",
    # variable_id=["rsds", "prc", "tasmin", "tasmax", "hur", "ua", "va"], # Variables for BFOLDS added
    variable_id=["rsds", "prc", "tasmin", "tasmax"]
)

# Making a dictionnary with the path to then access the data in xarray with OpenDAP,
# we can see that the function only return one single .nc file per dataset, for a single
# time period. The time period for each variable seems to be pretty random.
paths = cat.to_path_dict(prefer_streaming=True)
# print(paths)

   Searching indices:   0%|          |0/2 [       ?index/s]

Get file information:   0%|          |0/2 [       ?index/s]

In [7]:
# Now, we're going to loop around the OpenDAP paths and extract the data for the years we want and in
# the polygon we defined earlier, thanks to a custom function.
# WARNING : you might get errors if there are paths in the "paths" object that we created just before that
# are not OpenDAP URL paths. It might happen since the to_path_dict function of intake_esgf is still experimental.
# If this happens, you need to clean the dictionnary to only have the OpenDAP URLs you need.
from functionsForCalibration import *

# We initialize the dictionnary that are going to fill
# A key = one variable. The value will be a dataframe filled with the data from the variable.
dictOfDataFrames = dict()

for variable in paths.keys():
    # Since the data is often fragmented accross several .nc files that correspond to different ranges of year
    # for the data, we need to first initialize a panda dataframe, and then add rows to it.
    # That's what we do here.
    variableInitialized = False
    for path in paths[variable]:
        if not variableInitialized:
            dictOfDataFrames[variable] = GetDataFromESGFdataset(path,
                                            yearStart = 1950,
                                            yearStop = 2010,
                                            pathOfShapefileForSubsetting = "./ReferencesAndData/SpatialBoundaries/ChapleauBoundariesClimate.shp",
                                            nameOfVariable = variable,
                                            verbose = False,
                                            enableWarnings = False)
            if not dictOfDataFrames[variable].empty:
                variableInitialized = True
        else:
            additionalData = GetDataFromESGFdataset(path,
                                            yearStart = 1950,
                                            yearStop = 2010,
                                            pathOfShapefileForSubsetting = "./ReferencesAndData/SpatialBoundaries/ChapleauBoundariesClimate.shp",
                                            nameOfVariable = variable,
                                            verbose = False,
                                            enableWarnings = False)
            # The complex thing in pd.concat is used to deal with cases when GetDataFromESGFdataset returned an empty dataframe,
            # which happens when the .nc file didn't corresponded to the years we wanted.
            dictOfDataFrames[variable] = pd.concat([dictOfDataFrames[variable], additionalData if not additionalData.empty else None], ignore_index=True)
        # We sort the values by day, because they might be out of order.
        dictOfDataFrames[variable] = dictOfDataFrames[variable].sort_values(['year', 'month', 'day'])
        # print(dictOfDataFrames[variable])

In [9]:
# Now that we have the raw data accross several 100x100km grid cells for all variables necessary for PnET,
# we simply have to transform the measures if needed (e.g. PAR into umol/m2.s for the right radiation range instead of rsds which is W/m2)
# and then put everything in the right format for the LANDIS-II v8 climate library, and finally output things as .csv
# for latter use
import copy

##################################
# Restricting data to only one grid cell
# (since we will only simulate on cell in our monocultures)
##################################

# We get the unique coordinates in the dataframe
uniqueCoordinatesInDataFrames = extract_unique_coordinates(dictOfDataFrames["rsds"])
# Then, we choose the cell that is closest to the centroid of our chapleau region we've been using
bestCoordinatePair = find_closest_coordinate_to_polygon_center("./ReferencesAndData/SpatialBoundaries/ChapleauBoundariesClimate.shp",
                                                                uniqueCoordinatesInDataFrames)
# Then, we remove all references to other coordinate pairs
# Extract the latitude and longitude from the target coordinate
target_lat, target_lon = bestCoordinatePair
dictOfDataFramesSubset = copy.deepcopy(dictOfDataFrames)
for variable in dictOfDataFramesSubset.keys():
    # Create a mask for rows matching the target coordinate
    mask = (dictOfDataFramesSubset[variable]['lat'] == target_lat) & (dictOfDataFramesSubset[variable]['lon'] == target_lon)
    dictOfDataFramesSubset[variable] = dictOfDataFramesSubset[variable][mask]

# Now, we put all dataframes together in a single dataframe
mergedDataframe = pd.merge(dictOfDataFramesSubset["rsds"], dictOfDataFramesSubset["prc"], on = ["lat", "lon", "year", "month", "day"], how='left')
mergedDataframe = pd.merge(mergedDataframe, dictOfDataFramesSubset["tasmax"], on = ["lat", "lon", "year", "month", "day"], how='left')
mergedDataframe = pd.merge(mergedDataframe, dictOfDataFramesSubset["tasmin"], on = ["lat", "lon", "year", "month", "day"], how='left')

# WARNING : prc values are often 0. It's because it's daily precipitation data, it's normal !
# print(mergedDataframe)

##################################
# Transforming units (important !)
# And also variable names, to fit what is needed for the
# LANDIS-II v8 climate library
##################################

# tasmin and tasmax are in kelvin. We put then in celcius.
mergedDataframe['Tmin'] = mergedDataframe['tasmin'].apply(kelvin_to_celsius)
mergedDataframe['Tmax'] = mergedDataframe['tasmax'].apply(kelvin_to_celsius)

# Precipitations are in kg m-2 s-1 (see ESGF metagrid metadata for validation : https://esgf.nci.org.au/search?project=CMIP6&activeFacets=%7B%22nominal_resolution%22%3A%22100+km%22%2C%22frequency%22%3A%22day%22%2C%22experiment_id%22%3A%22historical%22%2C%22source_id%22%3A%22TaiESM1%22%2C%22variable_id%22%3A%22prc%22%7D)
# Transforming precipitations from their original unit - kg m-2 s-1 - into cm per day
mergedDataframe['precip'] = mergedDataframe['prc']*8640 # Going from seconds to days (86400 seconds in a day) and from mm (kg m-2) into cm (/10)

# For solar radiation : we first need to transform it from daily average to daytime average.
# Daily is on 24h (includes night); daily is average from sunrise to sunset.
# The data pulled from ESGF is daily; it has been validated to me in an email exchange by 
# Gary Strand, data maintener of the dataset i used at first : http://doi.org/10.22033/ESGF/CMIP6.10071
# We do that through a function that will calculate daytime hours based on the day of the year and the latitude/longitude.
mergedDataframe["rsds_daytimeAverage"] = [daylight_hour_average(row["rsds"], row["lat"], row["lon"], row["year"], row["month"], row["day"]) for index, row in mergedDataframe.iterrows()]

# Transforming downwelling shortwave radiation from W/m2 to umol.m2/s-1
# Based on the PnET User Guide's instructions
# Downwelling shortwave radiation (rsds which we have here) is often refered as global solar radiation.
# See https://library.wmo.int/viewer/68695/?offset=3#page=298&viewer=picture&o=search&n=0&q=shortwave . But other references exist.
# So, Downwelling shortave radiation is for wavelengths of 0.2–4.0 μm; PAR is for 0.4–0.7 μm.
# As such, to convert our Downwelling Shortwave Radiation in W/m2 to PAR in umol.m2/s-1, 
# we must multiply it by 2.02 as indicated in the user guide of PnET Succession.
mergedDataframe['PAR'] = mergedDataframe['rsds']*2.02 # Going from seconds to days (86400 seconds in a day)

# We remove the old columns
mergedDataframe.drop('tasmin', axis=1, inplace=True)
mergedDataframe.drop('tasmax', axis=1, inplace=True)
mergedDataframe.drop('prc', axis=1, inplace=True)
mergedDataframe.drop('rsds', axis=1, inplace=True)
mergedDataframe.drop('rsds_daytimeAverage', axis=1, inplace=True)

# We print the result
# print(mergedDataframe)

##################################
# Giving the dataframe the right columns
# for the LANDIS-II v8 climate library
##################################

# Create a unique identifier for each lat/lon pair
mergedDataframe['lat_lon'] = mergedDataframe['lat'].astype(str) + '_' + mergedDataframe['lon'].astype(str)

# Melt the dataframe to convert variables into rows
melted_df = pd.melt(
    mergedDataframe, 
    id_vars=['year', 'month', 'day', 'lat_lon'],
    value_vars=['Tmin', 'Tmax', "PAR", "precip"],
    var_name='Variable',
    value_name='value'
)

# Pivot the table to get lat_lon as columns (which can then be used as ecoregions)
result_df = melted_df.pivot_table(
    index=['year', 'month', 'day', 'Variable'],
    columns='lat_lon',
    values='value'
).reset_index()

# Convert the column index from MultiIndex to regular Index
result_df.columns.name = None
result_df = result_df.sort_values(by=['Variable', 'year', 'month', 'day'])

# We replace the name of the column that has the lat_lon pairing into an 
# ecoregion name (the one used in our PnET one cell scenario, eco1)
result_df = result_df.rename(columns={result_df.columns[-1]: 'eco1'})
# print(result_df)

# We print as .csv
result_df.to_csv("./ReferencesAndData/Climate Data/dataFrameClimate_historicalDaily.csv", sep=',', index=False, encoding='utf-8')

In [10]:
# We finish by averaging the values (which is recommanded by the PnET User to use in the first steps of the calibration,
# to have a very mild and stable climate with no extremes.
# As a result, we will still have a climate stream with daily data from 1950 to 2010,
# but every january 1st data for a given variable will the same, and will be the average of all january 1st in the original
# non-averaged data, etc.
from functionsForCalibration import *

dataFrameClimate_historicalDaily = pd.read_csv("./ReferencesAndData/Climate Data/dataFrameClimate_historicalDaily.csv")

dataFrameClimate_historicalAveraged = transform_to_historical_averages_vectorized(dataFrameClimate_historicalDaily)

# print(dataFrameClimate_historicalDaily)
# print(dataFrameClimate_historicalAveraged)

dataFrameClimate_historicalAveraged.to_csv("./ReferencesAndData/Climate Data/dataFrameClimate_historicalAveraged.csv", sep=',', index=False, encoding='utf-8')

### Climate source for ☁️ CO2 concentrations

PnET-Succession needs CO2 concentrations in the air to simulation vegetation dynamic. However, it doesn't seem to be a variable that is generated by the GCMs of the CMIP6. It looks like it's because CO2 concentration are often assumed homogeneous on earth since it's a well-mixed gas, because it's an intrant/restriction to the models rather than an output, etc.

Still, there are local variations around the atmosphere in CO2 concentrations (see [here](https://earthobservatory.nasa.gov/images/82142/global-patterns-of-carbon-dioxide)). The differences are small, and it seems like global values can be used (see [here](https://earthobservatory.nasa.gov/blogs/earthmatters/2016/12/05/reader-question-does-co2-disperse-evenly-around-the-earth/)). But I'm choosing here to try to get more nuanced data.

Data from https://www.nature.com/articles/s41597-022-01196-7#Sec5 seems very promising : available at https://zenodo.org/records/5021361, not too big, allows for local variations in CO2 concentration. Would need to script the download and treatment of the files here. Timestep is monthly, but we can downscale it to daily and still at least have some monthly variation. Goes to 2150, so perfect for our uses.

Justification for using the dataset :

- We know that this CO2 dataset does not come from the same GCM that generated our climate data, and there could be variability between the variability of climate variables in one dataset versus the climate variables in the models that generated these CO2 estimates at each given timestep.
- However, We thought it better to use this dataset to approximate CO2 spatial and temporal variations in our simulations rather than using latitudinal or global averages. 

In [11]:
# First, we download the CO2 data from https://zenodo.org/records/5021361 

from functionsForCalibration import *
import pandas as pd
import urllib.request
import subprocess
import os

# We use the zenodo_get library to download files from Zenodo very fast; other methods tend to be slow.
# See https://github.com/dvolgyes/zenodo_get . It is installed in the Docker image.
# WARNING : will take around 1.5GB of space for the data from 1950 to 2150 for a full climate scenario.

# We download the file with historical data
os.system("cd ReferencesAndData && zenodo_get 5021361 -g CO2_1deg_month_1850-2013.nc")

# We can also download data for future conditions, like this :
# os.system("cd ReferencesAndData && zenodo_get 5021361 -g CO2_SSP126_2015_2150.nc")

Title: Global monthly distributions of atmospheric CO2 concentrations under the historical and future scenarios
Keywords: Atmospheric chemistry, Atmospheric dynamics, Climate and Earth system modelling
Publication date: 2021-06-23
DOI: 10.5281/zenodo.5021361
Total size: 1.0 GB

Link: https://zenodo.org/records/5021361/files/CO2_1deg_month_1850-2013.nc   size: 1.0 GB

Checksum is correct. (b11db6cc374ce23fd41b3b078cc8ebdd)
All files have been downloaded.


0

In [1]:
from functionsForCalibration import *

historical_ds = xr.open_dataset("./ReferencesAndData/CO2_1deg_month_1850-2013.nc")
# future_ds = xr.open_dataset("./ReferencesAndData/CO2_SSP126_2015_2150.nc")

# Shapefile to select where we take our values; if several climate cells are in the shapefile,
# then the CO2 values in all cells will be averaged (see function process_co2_data)
shapefile = "./ReferencesAndData/SpatialBoundaries/ChapleauBoundariesClimate.shp"

# We load the dataframe
# We prepare the dataframe in a format that matches the older function I made
# When I was using monthly data and LANDIS-II v7.
# We load the file created previously
df_climate = pd.read_csv("./ReferencesAndData/Climate Data/dataFrameClimate_historicalDaily.csv")
# Convert to year month day to string
df_climate['date_string'] = (df_climate['year'].astype(str) + '-' + 
                     df_climate['month'].astype(str) + '-' + 
                     df_climate['day'].astype(str))
# Get unique dates
uniqueDates = df_climate['date_string'].unique()
# Create the new dataframe
dfToFill = pd.DataFrame(columns=["year", "month", "day", "Variable"])
# Put the dates in
dfToFill["date_string"] = uniqueDates
# extract the dates correctly
dfToFill['year'] = dfToFill["date_string"].str.split('-').str[0].astype(int)
dfToFill['month'] = dfToFill["date_string"].str.split('-').str[1].astype(int)
dfToFill['day'] = dfToFill["date_string"].str.split('-').str[2].astype(int)
# Add variable name
dfToFill["Variable"] = ["CO2"] * len(dfToFill)
# Remove the date_string column
dfToFill.drop('date_string', axis=1, inplace=True)
df_climate.drop('date_string', axis=1, inplace=True)
# Now, the dataframe is ready to be filled with CO2 values with the old function !
# print(dfToFill)

dfToFill = process_co2_data(historical_ds, shapefile, dfToFill)

# Display the updated dataframe
print(dfToFill.head())
print(dfToFill.tail())

No points found inside the polygon for historical data. Finding closest point...
Closest point to polygon centroid for historical data: Lon=-83.5, Lat=47.5
   year  month  day Variable  CO2_Concentration
0  1950      1    1      CO2         317.369324
1  1950      1    2      CO2         317.369324
2  1950      1    3      CO2         317.369324
3  1950      1    4      CO2         317.369324
4  1950      1    5      CO2         317.369324
       year  month  day Variable  CO2_Concentration
22260  2010     12   27      CO2         399.926788
22261  2010     12   28      CO2         399.926788
22262  2010     12   29      CO2         399.926788
22263  2010     12   30      CO2         399.926788
22264  2010     12   31      CO2         399.926788


In [4]:
# Finally, we add this CO2 data to the previous data, and re-average it if needed

# First, we have to rename the CO2 column in the dataframe we just filled so that it corresponds to an ecoregion now
# (this is due to the difference in the climate library formats with LANDIS-II v8 and the old functions I was using before switching)
# The column name CO2_Concentration is created by the function process_co2_data
dfToFill = dfToFill.rename(columns={'CO2_Concentration': 'eco1'})

# We concatenate the dataframes
df_climate = pd.concat([df_climate, dfToFill], ignore_index=True)

# We check if needed
print(df_climate)

# We average
dataFrameClimate_historicalAveraged = transform_to_historical_averages_vectorized(df_climate)
# We save everything
dataFrameClimate_historicalAveraged.to_csv("./ReferencesAndData/Climate Data/dataFrameClimate_historicalAveraged.csv", sep=',', index=False, encoding='utf-8')

# Now that everything is done, we can remove the CO2 files downloaded
os.remove("./ReferencesAndData/CO2_1deg_month_1850-2013.nc")
# os.remove("./ReferencesAndData/CO2_SSP126_2015_2150.nc")

        year  month  day Variable        eco1
0       1950      1    1      PAR   37.535774
1       1950      1    2      PAR  108.757004
2       1950      1    3      PAR   72.965851
3       1950      1    4      PAR  110.865730
4       1950      1    5      PAR   92.960808
...      ...    ...  ...      ...         ...
155850  2010     12   27      CO2  399.926788
155851  2010     12   28      CO2  399.926788
155852  2010     12   29      CO2  399.926788
155853  2010     12   30      CO2  399.926788
155854  2010     12   31      CO2  399.926788

[155855 rows x 5 columns]


## Soils

Gustafon recommands SILO (Silty Loam/Silt Loam) soil from the default PnET Succession SaxtonAndRawls file (`C:\Program Files\LANDIS-II-v7\extensions\Defaults\SaxtonAndRawlsParameters.txt`) as a soil that retains water well. This is because the first calibration step should use an "ideal soil" where water is well-retained so that lack of water (or watterlogging) does not influence growth at this calibration step.

We can use the SILO soil type easily by modifying the SoilType of `eco1` in the PnET One Cell Scenario that we will use (see `SimulationFiles/PnETGitHub_OneCellSim_v8/`). So, nothing more to do here.

## CLUES FOR FUTURE DATA

- Alex Chubaty might have scripts to generate initial data in BC and Alberta, and even other provinces ?
- Caren Dymonc for initial data in BC ?
- Yan Boulanger : BioSIM climate data for small scale/downscaled climate data for PnET + BFOLDS ? Needed so that we can have PnET Climate input that change according to slope and topography in particular. Will need to create many smaller ecoregions that are combo of soils + climate "zones" in the area + classes of slopes/aspect. BioSIM might help in generating those things.